In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

In [2]:
import tensorflow as tf
from sklearn.metrics import classification_report

2024-03-22 17:12:05.619639: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-22 17:12:05.619660: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-22 17:12:05.620155: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [88]:
def print_classification_report(model, dataset):
    labels = []
    predictions = []
    
    for X, y in dataset:
        labels.extend(y.numpy().tolist())
        y_pred = tf.argmax(model.predict(X, verbose=0), axis=1).numpy().tolist()
        predictions.extend(y_pred)

    print(classification_report(labels, predictions, target_names=dataset.class_names))

In [3]:
seed = 42
dataset_path = '/home/deeplearning/AI/datasets/TrashBox'
epochs = 50
batch_size = 32

In [96]:
train_ds_raw = tf.keras.utils.image_dataset_from_directory(dataset_path + '/train/sub_classes', batch_size=batch_size)
train_ds = train_ds_raw.prefetch(tf.data.AUTOTUNE)
val_ds = tf.keras.utils.image_dataset_from_directory(dataset_path + '/val', batch_size=batch_size)
test_ds = tf.keras.utils.image_dataset_from_directory(dataset_path + '/test', batch_size=batch_size)

Found 14276 files belonging to 25 classes.
Found 1774 files belonging to 25 classes.
Found 1802 files belonging to 25 classes.


In [97]:
model_path = 'models/efficientnet_v2_trash_box.keras' #model_checkpoint_callback.filepath
model = tf.keras.models.load_model(model_path)

In [98]:
print("Train evaluation")
model.evaluate(train_ds)

Train evaluation
447/447 [==============================] - 16s 32ms/step - loss: 0.0015 - accuracy: 0.9994


[0.0015306634595617652, 0.9993695616722107]

In [99]:
print("Val evaluation")
model.evaluate(val_ds)

Val evaluation
56/56 [==============================] - 2s 33ms/step - loss: 0.3656 - accuracy: 0.9295


[0.3655843436717987, 0.9295377731323242]

In [100]:
print("Test evaluation")
model.evaluate(test_ds)

Test evaluation
57/57 [==============================] - 2s 26ms/step - loss: 0.4004 - accuracy: 0.9206


[0.4003887474536896, 0.9206437468528748]

In [102]:
print_classification_report(model, train_ds_raw)

                     precision    recall  f1-score   support

      beverage cans       1.00      1.00      1.00       800
          cardboard       1.00      1.00      1.00      1929
     cigarette butt       1.00      1.00      1.00       405
 construction scrap       1.00      1.00      1.00       431
  electrical cables       1.00      1.00      1.00       554
   electronic chips       1.00      1.00      1.00       492
              glass       1.00      1.00      1.00      2022
             gloves       1.00      1.00      1.00       354
            laptops       1.00      1.00      1.00       400
              masks       1.00      1.00      1.00       400
          medicines       1.00      1.00      1.00       405
   metal containers       1.00      1.00      1.00       404
         news paper       1.00      0.99      0.99       160
other metal objects       1.00      1.00      1.00        33
other paper objects       1.00      1.00      1.00       849
         paper cups    

In [103]:
print("Val classification report")
print_classification_report(model, val_ds)

Val classification report
                     precision    recall  f1-score   support

      beverage cans       0.91      0.93      0.92       100
          cardboard       0.96      0.93      0.94       241
     cigarette butt       0.98      1.00      0.99        50
 construction scrap       0.91      0.81      0.86        53
  electrical cables       0.91      0.93      0.92        67
   electronic chips       0.88      0.83      0.85        63
              glass       0.97      0.94      0.96       252
             gloves       0.88      1.00      0.94        44
            laptops       0.84      0.86      0.85        49
              masks       0.94      0.92      0.93        49
          medicines       0.96      0.96      0.96        50
   metal containers       0.84      0.96      0.90        50
         news paper       0.95      0.95      0.95        20
other metal objects       1.00      0.50      0.67         4
other paper objects       0.93      0.95      0.94       1

In [104]:
print("Test classification report")
print_classification_report(model, test_ds)

Test classification report
                     precision    recall  f1-score   support

      beverage cans       0.88      0.94      0.91       100
          cardboard       0.92      0.93      0.92       242
     cigarette butt       1.00      1.00      1.00        52
 construction scrap       0.87      0.85      0.86        55
  electrical cables       0.94      0.87      0.90        70
   electronic chips       0.89      0.76      0.82        62
              glass       0.95      0.96      0.96       254
             gloves       0.96      0.98      0.97        45
            laptops       0.82      0.82      0.82        50
              masks       0.94      0.96      0.95        50
          medicines       0.89      0.96      0.93        52
   metal containers       0.87      0.90      0.88        51
         news paper       0.80      0.80      0.80        20
other metal objects       0.00      0.00      0.00         5
other paper objects       0.91      0.92      0.91       

In [105]:
other_metal_objects_path = dataset_path + '/test/other metal objects'
os.listdir(other_metal_objects_path)

['metal_objects 7.jpg',
 'metal_objects 5.jpg',
 'metal_objects 3.jpg',
 'metal_objects 4.jpg',
 'metal_objects 41.jpg']

In [137]:
def decode_img(img):
  # Convert the compressed string to a 3D uint8 tensor
  img = tf.io.decode_jpeg(img, channels=3)
  # Resize the image to the desired size
  return tf.image.resize(img, [300, 300])

In [138]:
def process_path(file_path):
  # Load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img

In [139]:
list_ds = tf.data.Dataset.list_files(dataset_path + '/test/other metal objects/*')

In [140]:
img_ds = list_ds.map(process_path).map(tf.keras.applications.efficientnet.preprocess_input).batch(1)

In [141]:
for X in img_ds:
    print(X.shape)
    y_preds = tf.argmax(model.predict(X, verbose=0), axis=1).numpy()[0]
    print(y_preds, test_ds.class_names[y_preds])

(1, 300, 300, 3)
13 other metal objects
(1, 300, 300, 3)
0 beverage cans
(1, 300, 300, 3)
11 metal containers
(1, 300, 300, 3)
0 beverage cans
(1, 300, 300, 3)
0 beverage cans


In [81]:
print(classification_report([13]*5, [0,13,0,13,0]))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
          13       1.00      0.40      0.57         5

    accuracy                           0.40         5
   macro avg       0.50      0.20      0.29         5
weighted avg       1.00      0.40      0.57         5



/home/deeplearning/AI/projects/open-waste/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/deeplearning/AI/projects/open-waste/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/deeplearning/AI/projects/open-waste/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{m

In [115]:
for X, y in test_ds:
    y_preds = tf.argmax(model.predict(X, verbose=0), axis=1).numpy().tolist()
    y = y.numpy().tolist()
    for i in zip(y, y_preds):
        if i[0] == 13:
            print(X.shape, i[0], i[1])

(32, 256, 256, 3) 13 11
(32, 256, 256, 3) 13 0
(32, 256, 256, 3) 13 0
(32, 256, 256, 3) 13 0
(32, 256, 256, 3) 13 0
